## What is Data Science?

The hot takes via [Twitter](https://twitter.com/mcmoots/status/429318287864770560):

- "A data scientist is a statistician who lives in San Francisco." 
- "A data scientist is a business analyst who lives in New York."
- "Data Science is statistics on a Mac."

A more idealistic view:

![Venn Diagram of Data Science](https://raw.githubusercontent.com/jakevdp/PythonDataScienceHandbook/8a34a4f653bdbdc01415a94dc20d4e9b97438965/notebooks/figures/Data_Science_VD.png)

"...think of data science not as a new domain of knowledge to learn, but a new set of skills that you can apply within your current area of expertise." - [Jake VanderPlas](http://vanderplas.com/), Director of Open Software at the University of Washington's eScience Institute.

## So what's the plan?

We're going to combine a touch of stats, a hint of hacking, and a pinch of substantive knowledge about policing to analyze a real data set, collected by researchers at the [Stanford Open Policing Project](https://openpolicing.stanford.edu/).

On a typical day in the United States, police officers make more than **50,000 traffic stops**. Stanford has collected over 200 million records from dozens of state and local police departments across the country in an effort to create a centralized repository detailing interactions between civilians and police. 

We're going to look at a subset of this data: **Maryland in 2013-2014, excluding the City of Baltimore.**

Why just a subset? It keeps the size manageable, plus we avoid the sparse sections of the MD dataset. From Stanford:

> The [Maryland] data is very messy. It comes from three different time periods: 2007, 2009-2012, 2013-2014. They all have different column and slightly different conventions of how things are recorded. We attempted to standardize the fields as much as possible.
Time resolution of the data varies by year. Prior to 2013, data is reported annually. From 2013 onward, data is reported daily. So stop dates prior to 2013 are not precise to the nearest day and are just reported as Jan 1.
Counties were mapped by running the police departments in the Agency field through Google's geocoder, but this does not work for state patrol stops, for which we have no county information.

### 1. Import packages
See all the "as ..." contructs? They're just aliasing the package names.

That way we can call methods like plt.plot() instead of matplotlib.pyplot.plot().

The aliases used here are conventional in the Python data community.

In [ ]:
%matplotlib inline 
# Above line is not necessary in latest version of Notebook, but in older versions
# ensures that plots are displayed inline, not in a separate window

import numpy as np # imports a fast numerical programming library
import matplotlib as mpl # this actually imports matplotlib
import matplotlib.pyplot as plt # sets up plotting under plt
import pandas as pd #lets us handle data as dataframes
import seaborn as sns #sets up styles and gives us more plotting options

In [ ]:
# Boilerplate that sets some display options. The defaults are fine too!
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 25)
pd.set_option('display.notebook_repr_html', True)

In [ ]:
# This is an f-string. If it doesn't work, you're running a Python version below 3.6. Consider upgrading!
print(f"Numpy Version: {np.__version__}, Pandas Version: {pd.__version__}")

### 2. Import data

Why `low_memory = False`? Inferring datatypes is very memory intensive and we're only specifying some of the dtypes during import. This suppresses a warning about memory use when we load data without specifying all dtypes.

In [ ]:
md = pd.read_csv('md_traffic.csv', low_memory = False, dtype={
    'subject_age': 'float64',
    'subject_race': 'category',
    'subject_sex': 'category',
    'arrest_made': 'bool',
    'outcome': 'category',
    'search_conducted': 'bool',
    'reason_for_arrest': 'category',
    })

##### Aside: Did you know you can measure execution time of small code snippets using the `%timeit` command?

Notice that `timeit` executes the code several times and takes an average of the runs.
Read more about it in the [official documentation](https://docs.python.org/3.7/library/timeit.html).

In [ ]:
import random
%timeit array = [random.randint(0, 10) for i in range(100000)]
%timeit values = np.random.randint(0, 10, 100000)

##### Aside #2: Did you know you can access a function's signature & docstring by adding a "?" after a command. E.g. `pd.read_csv?`
It's the equivalent of `print(pd.read_csv.__doc__)` but `?` is a shortcut in the IPython environment.

In [ ]:
# 'read_csv' is a Crime Against Nature masquerading as a function, but it gets the job done.
pd.read_csv?

### 3. Basic Data Exploration

In [ ]:
# Let's look at the shape of this data.
md.shape

QUESTION: Can this CSV file be opened in Excel? What about Apple's Numbers App?

In [ ]:
# Check the index. We see a RangeIndex because we did not specify a column to be an index when reading the csv.
md.index

In [ ]:
# Check the columns.
md.columns

In [ ]:
md.dtypes

In [ ]:
# I prefer .info() to .dtypes as the former also tells us how many null points we're dealing with
md.info()

In [ ]:
# Peak at the data
md.head(4)

What can we discern just from this look at the data? What about all the NaNs?

We may want to set `raw_row_number` as the index.

In [ ]:
#Verify_integrity is set to false by default. Setting to True ensures a unique index.
md.set_index("raw_row_number", verify_integrity=True, inplace=True)

In [ ]:
md.head()
# What did the first driver get cited for?

We can access individual columns through dot-attribute notation (`md.subject_sex`) or through
getitem dictionary-style notation (`md['subject_sex']`).

QUESTION: Why might the latter be preferable even as the former is somewhat more natural to write?

The return value is a Pandas Series; basically a one-column DataFrame.

In [ ]:
# Attribute notation
md.subject_sex.head()

In [ ]:
#getitem syntax (uses __.getitem__)
md['outcome'].head()

In [ ]:
# Access multiple columns with a comma
md[['department_name', 'citation_issued', 'reason_for_stop']].head()

You can index the rows by using the **loc** and **iloc** accessors.

`loc` does *label-based* indexing.

`iloc` performs *integer-based* indexing.

You can use a comma separated list to access multiple fields at the same time.

Both support the standard Python slicing operations `[start:end:step]`

In [ ]:
#get the rows for index IDs 3237323 and 2312594
md.loc[[3237323,2312594]]

In [ ]:
#get every tenth row starting from 4100, until 4150.
md.iloc[4100:4150:10]

In [ ]:
# show the first 5 rows and the first 5 columns
md.iloc[:5, :5]

In [ ]:
# show the all the rows up to index 2312588 and the columns named 'subject_sex' and 'warning_issued'
md.loc[:2312588, ['subject_sex', 'warning_issued']]

### 4. Dealing with Data Types and NaN Cleaning 
![Office Space Meme](https://media.makeameme.org/created/i-was-told-592f12.jpg)


We might be worried about the large number of NaNs in the dataset. Another way to quantify our null fields is by summing the result of applying `isnull()` to the dataframe.

In [ ]:
# Why does this work? Think about what True and False are equivalent to.
md.isnull().sum()

But are the null fields really a problem? It depends. If `reason_for_search` is ALWAYS null, we have a problem, but if it's only null when `search_conducted` is False, we're in business.

In [ ]:
md.search_conducted == True

In [ ]:
md[md.search_conducted == True].isnull().sum()

Looks like `reason_for_search` is null only 56 times when a search has been conducted. An enterprising journalist might want to look at these 56 cases to see if there's anything fishy or if it's just a bookkeeping error (likely!)

If we hadn't done data conversions at import, we could do so now:

Convert column "subject_age" to float64 dtype and "subject_race" to categorical type
`md = md.astype({"subject_age": 'float', "subject_race": 'category'})`

In [ ]:
#may want to use string methods to normalize string objects
md.location.str.lower().head()

### 5. Exploring the Data More Deeply

Here, the memes stop and we have to be more sober.

Let's look at the kinds of values we find in various columns.

In [ ]:
# we can also normalize this
md.subject_sex.value_counts()

In [ ]:
# can apply functions to columns
md.subject_age.mean()

###### QUESTION: Who is the youngest person in this data set? What were they cited for? Does the citation pass the smell test?

In [ ]:

# Your Code Here

In [ ]:
# Don't execute until ready to see the answer
%load solution/youngest.py


There are dozens of ways to plot age data. Let's look at two. For more on what KDE means, see [Kernel Density Estimation](https://en.wikipedia.org/wiki/Kernel_density_estimation).

In [ ]:
# Seaborn is very pretty!
sns.distplot(md.subject_age[md.subject_age.isnull()==False], kde= True, bins = 15);

In [ ]:
# Pandas (which uses Matplotlib under the hood) is less pretty by default, but very convenient
md.subject_age.plot(kind='hist');

### 6. Exploring Race and Policing

In [ ]:
md.subject_race.value_counts()

In [ ]:
with pd.option_context('max_rows', 210):
    print(md.department_name.value_counts())

We have to establish some baseline to determine if the proportion of White, Black, Hispanic, and Asian drivers being pulled over is disproportionate to their share of the population. Since we don't have good data for the whole state, let's compare select counties' traffic stops to their Census populations.

In [ ]:
bcpd = md[md.department_name == 'Baltimore County Police Department']
bcpd_by_race = bcpd.subject_race.value_counts(normalize=True)
bcpd_by_race

We note that Black motorists represent 49% of the people pulled over by the Baltimore County Police Department in 2013 - early 2014. What about Montgomery?

In [ ]:
mcpd = md[(md.department_name == 'MONTGOMERY') | (md.department_name == 'Montgomery County Police Department') |(md.department_name == 'Montgomery County Sherrif\'s Office') ]
mcpd_by_race = mcpd.subject_race.value_counts(normalize=True)
mcpd_by_race

##### EXERCISE: Create a new data frame for PG County 

In [ ]:
# your code here


In [ ]:
# Don't execute until ready to see the answer
%load solution/pg.py


See [Census Data](https://www.census.gov/quickfacts/fact/table/princegeorgescountymaryland,baltimorecountymaryland,montgomerycountymaryland,US/PST045218)

In [ ]:
baltimore_co_census = pd.Series([.572, .292,.055 ,.064,], index=['white', 'black', 'hispanic', 'asian/pacific islander'])
moco_census = pd.Series([.438, .197, .196, .156,], index=['white', 'black', 'hispanic', 'asian/pacific islander'])

In [ ]:
moco_benchmark = pd.concat([mcpd_by_race, moco_census], keys=['police_stops', 'moco_census'], axis = 1)
moco_benchmark

Let’s do the same sort of benchmark comparison for search and frisk rates. We can use the stopped population as our baseline, defining search rate to be the proportion of stopped people who were subsequently searched, and frisk rate as proportion of stopped people who were subsequently personally searched.

We'll want to use pandas `GroupBy` functionality to implement the `split-apply-combine` pattern.

The idea here is that we **split** the data by some key or set of keys then **apply** a function to each group and then **combine** the outputs back into a single DataFrame.

In [ ]:
# In MoCo, how often are Hispanic drivers searched after being pulled over? White drivers? Asian drivers?
mcpd.groupby('subject_race').search_conducted.value_counts(normalize=True)

In [ ]:
# How often are drivers frisked during a stop?
mcpd[mcpd.search_conducted == True].groupby('subject_race').search_person.value_counts(normalize=True)

In [ ]:
# What about statewide?
md.groupby('subject_race').search_conducted.value_counts(normalize=True)

In [ ]:
md[md.search_conducted == True].groupby('subject_race').search_person.value_counts(normalize=True)

![https://imgs.xkcd.com/comics/correlation.png](https://imgs.xkcd.com/comics/correlation.png)

**"Correlation doesn't imply causation, but it does waggle its eyebrows suggestively and gesture furtively while mouthing 'look over there'."** - [XKCD 552](https://www.explainxkcd.com/wiki/index.php/552:_Correlation)

To get closer to causality, we can use the "outcome test" in which a lower search  success rate for one group relative to another is seen as evidence of bias against that group, as it suggests a lower evidentiary bar was applied when making search decisions.

In [ ]:
# pay close attention to the figure for Hispanics
md[md.search_conducted == True].groupby('subject_race').contraband_found.value_counts(normalize=True)

In [ ]:
md[md.search_conducted == True].groupby('subject_race').outcome.value_counts(normalize=True)

In [ ]:
mcpd[mcpd.search_conducted == True].groupby('subject_race').outcome.value_counts(normalize=True)

### 7. Closing Remarks

## Attribution And Further Exploration

- Jake VanderPlas: [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/): Free, CC License, available as a series of Jupyter Notebooks. Chapter 3 has strong coverage of Pandas.
- [dataMontgomery](https://data.montgomerycountymd.gov/): Montgomery County, MD offers excellent datasets on various public services, including law enforcement. For example, its 1.4 million row/35 column dataset on traffic stops has data on vehicle make/model, city of origin of the driver, and exact geolocation of the stop.
- [The Stanford Open Policing Project](https://openpolicing.stanford.edu/): See the whole dataset, plus a tutorial on how to work with it (unfortunately in R)
- [Kaggle DataSets](https://www.kaggle.com/datasets): Collection of datasets, many amendable to machine learning techniques